In [1]:
import os, pathlib
os.chdir("/Users/gregcc/Documents/GitHub/SURP25/Microlensing")

In [2]:
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from ML import TwoLens1S
from ML import ThreeLens1S

In [3]:
import numpy as np

# ---------- Roman cadence helpers ----------
def roman_sample(t_days, y, cadence_min=12.1, exp_seconds=67, nsub=7, season_blocks=None):
    """Downsample a dense time series to Roman-like cadence with finite-exposure averaging."""
    t_days = np.asarray(t_days); y = np.asarray(y)
    cad_days = cadence_min / (60.0 * 24.0)
    exp_days = exp_seconds / 86400.0

    # Build sampling grid (respect optional observing seasons)
    if season_blocks is None:
        t0, t1 = t_days.min(), t_days.max()
        start = np.ceil((t0 - t0)/cad_days)*cad_days + t0
        ts = np.arange(start, t1 + 0.5*cad_days, cad_days)
    else:
        chunks = []
        for a, b in season_blocks:
            a = max(a, t_days.min()); b = min(b, t_days.max())
            if a < b: chunks.append(np.arange(a, b + 0.5*cad_days, cad_days))
        if not chunks: return np.array([]), np.array([])
        ts = np.concatenate(chunks)

    # Boxcar average across the exposure using linear interpolation
    offs = np.linspace(-0.5*exp_days, 0.5*exp_days, nsub)
    ys = np.empty_like(ts, dtype=float)
    for i, t0 in enumerate(ts):
        subs = np.clip(t0 + offs, t_days[0], t_days[-1])
        ys[i] = np.interp(subs, t_days, y).mean()
    return ts, ys

def roman_gbt_ds_windows(t_start, n_seasons=6, season_len_days=72, gap_days=0):
    """Simple season windows maker (adjust gap_days if you want spacing)."""
    wins, t = [], float(t_start)
    for _ in range(n_seasons):
        wins.append((t, t + season_len_days))
        t += season_len_days + max(0, gap_days)
    return wins

# ---------- Your parameters ----------
q2 = 1e-3
q3 = 1e-5
s2 = 1.3
s3 = 1.3003845585056752
psi = 1.393457227228705
q4 = q3 + q2   # (not used below, but fine to keep for cross-checks)
u0_list = [-0.0445]

t0 = 0.0
tE = 20.0
rho = 0.0000018
rs = rho

secnum = 45
basenum = 2
num_points = 1000

# ---------- Build your model ----------
triple_model = ThreeLens1S(
    t0, tE, rho, u0_list,
    q2, q3, s2, s3, 357,  # alpha_deg = 357
    psi, rs, secnum, basenum, num_points
)

param = [
    np.log(triple_model.s2), np.log(triple_model.q2), triple_model.u0_list[0], triple_model.alpha_deg,
    np.log(triple_model.rho), np.log(triple_model.tE), triple_model.t0,
    np.log(triple_model.s3), np.log(triple_model.q3), triple_model.psi_rad
]

# ---------- Make a dense time grid ----------
# Prefer the class’s own dense grid if it exists; otherwise make ~30s sampling over ±2 tE:
if hasattr(triple_model, "highres_t"):
    t_dense = np.asarray(triple_model.highres_t, dtype=float)
else:
    dt_days = 30.0 / 86400.0
    t_dense = np.arange(t0 - 2*tE, t0 + 2*tE + dt_days, dt_days)

# Run VBM to set geometry & get photometry (even if you only need centroids)
mag, *_ = triple_model.VBM.TripleLightCurve(param, t_dense)

# Pull centroids & source positions from the first system (Einstein-radius units)
sys0 = triple_model.systems[0]
cent_x = np.asarray(sys0['cent_x'])
cent_y = np.asarray(sys0['cent_y'])
y1s    = np.asarray(sys0['y1s'])  # source x
y2s    = np.asarray(sys0['y2s'])  # source y

# Your convention from earlier (note the minus on x):
dx = -(cent_x - y1s)
dy =  (cent_y - y2s)
dr = np.hypot(dx, dy)   # total centroid shift

# ---------- Roman cadence & exposure averaging ----------
# Option A: entire time span at 12.1 min, 67 s
ts_dx, dx_s = roman_sample(t_dense, dx, cadence_min=12.1, exp_seconds=67)
_,     dy_s = roman_sample(t_dense, dy, cadence_min=12.1, exp_seconds=67)
dr_s = np.hypot(dx_s, dy_s)

# Option B (recommended): restrict to 6×72-day seasons (edit gaps if desired)
wins = roman_gbt_ds_windows(t_start=t_dense.min(), n_seasons=6, season_len_days=72, gap_days=0)
ts_dx_season, dx_s_season = roman_sample(t_dense, dx, cadence_min=12.1, exp_seconds=67, season_blocks=wins)
_,             dy_s_season = roman_sample(t_dense, dy, cadence_min=12.1, exp_seconds=67, season_blocks=wins)
dr_s_season = np.hypot(dx_s_season, dy_s_season)

# ---------- (Optional) convert to µas if you know theta_E ----------
# Set your Einstein radius on-sky (e.g., ~200 µas is common; replace with your event’s value)
thetaE_muas = None  # e.g., 200.0
if thetaE_muas is not None:
    dx_muas       = dx * thetaE_muas
    dy_muas       = dy * thetaE_muas
    dr_muas       = dr * thetaE_muas
    dx_s_muas     = dx_s * thetaE_muas
    dy_s_muas     = dy_s * thetaE_muas
    dr_s_muas     = dr_s * thetaE_muas
    dx_s_season_muas = dx_s_season * thetaE_muas
    dy_s_season_muas = dy_s_season * thetaE_muas
    dr_s_season_muas = dr_s_season * thetaE_muas

# ---------- (Optional) quick visualization ----------
try:
    import matplotlib.pyplot as plt

    plt.figure()
    plt.plot(t_dense, dr, lw=1, label='dense (Einstein radii)')
    plt.scatter(ts_dx, dr_s, s=8, label='Roman 12.1 min + 67 s')
    plt.xlabel('Time (days)'); plt.ylabel('Centroid shift (θ_E units)')
    plt.legend(); plt.title('Centroid shift with Roman cadence'); plt.show()

    # Trajectory in the astrometric plane
    plt.figure()
    plt.plot(dx, dy, lw=1, label='dense')
    plt.scatter(dx_s, dy_s, s=8, label='Roman sampled')
    plt.xlabel('Δx (θ_E)'); plt.ylabel('Δy (θ_E)')
    plt.axis('equal'); plt.legend(); plt.title('Astrometric trajectory'); plt.show()
except Exception as e:
    print("Plotting skipped:", e)


KeyboardInterrupt: 